In [ ]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle

In [17]:
num_vertices = 100

In [18]:
config = {
    'experiment_name': 'vertix_edge_training_l1_greedy',
    'device': 'cuda:0',  
    'is_overfit': False,
    'batch_size': 8,
    'resume_ckpt': None,
    'learning_rate': 5e-4,
    'max_epochs': 5000,
    'print_every_n': 100,
    'validate_every_n': 3200,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'train',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2,
    'train_pickle': './train_100_2.pickle',
    'val_pickle': './val_100_2.pickle'
}

In [13]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "val", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [14]:
dataset.filter_data()

Length of dataset: 32304
Filtering data ..


100%|████████████████████████████████████| 32304/32304 [00:57<00:00, 560.19it/s]

Length of dataset: 5474


In [15]:
with open('val_100_2.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from training import vertix_edge_train

vertix_edge_train.main(config)

Device: cuda:0
Length of dataset: 153528
Filtering data ..


100%|█████████████████████████████████| 153528/153528 [00:49<00:00, 3071.35it/s]


Length of dataset: 26196


100%|████████████████████████████████| 26196/26196 [00:00<00:00, 1130425.71it/s]


Class 03001627 has 4072 shapes
Class 04530566 has 2424 shapes
Class 04379243 has 6162 shapes
Class 03636649 has 1518 shapes
Class 02691156 has 2360 shapes
Class 02958343 has 3372 shapes
Class 04256520 has 4574 shapes
Class 02933112 has 1714 shapes
Length of dataset: 32304
Filtering data ..


100%|███████████████████████████████████| 32304/32304 [00:30<00:00, 1073.78it/s]


Length of dataset: 5474


100%|██████████████████████████████████| 5474/5474 [00:00<00:00, 1094096.74it/s]


Class 02691156 has 456 shapes
Class 04379243 has 1254 shapes
Class 02958343 has 700 shapes
Class 04256520 has 1016 shapes
Class 03636649 has 394 shapes
Class 02933112 has 360 shapes
Class 04530566 has 500 shapes
Class 03001627 has 794 shapes
[000/00099] train_loss: 0.223278, train_vertices_loss: 0.111578, train_edges_loss: 0.111700
[000/00199] train_loss: 0.178762, train_vertices_loss: 0.096839, train_edges_loss: 0.081923
[000/00299] train_loss: 0.176676, train_vertices_loss: 0.096535, train_edges_loss: 0.080142
[000/00399] train_loss: 0.176466, train_vertices_loss: 0.096446, train_edges_loss: 0.080020
[000/00499] train_loss: 0.171064, train_vertices_loss: 0.094326, train_edges_loss: 0.076738
[000/00599] train_loss: 0.185907, train_vertices_loss: 0.104821, train_edges_loss: 0.081085
[000/00699] train_loss: 0.174253, train_vertices_loss: 0.095535, train_edges_loss: 0.078718
[000/00799] train_loss: 0.177318, train_vertices_loss: 0.097195, train_edges_loss: 0.080122
[000/00899] train_loss

[002/02049] train_loss: 0.151204, train_vertices_loss: 0.079731, train_edges_loss: 0.071473
[002/02149] train_loss: 0.158463, train_vertices_loss: 0.082312, train_edges_loss: 0.076150
[002/02249] train_loss: 0.152317, train_vertices_loss: 0.079933, train_edges_loss: 0.072384
[002/02349] train_loss: 0.149944, train_vertices_loss: 0.078360, train_edges_loss: 0.071584
[002/02449] train_loss: 0.149126, train_vertices_loss: 0.077876, train_edges_loss: 0.071250
[002/02549] train_loss: 0.150185, train_vertices_loss: 0.078855, train_edges_loss: 0.071330


In [7]:
from inference.inference_vertix_edge import InferenceHandlerVertixEdgeModel

# create a handler for inference using a trained checkpoint
inferer = InferenceHandlerVertixEdgeModel('runs/vertix_edge_overfitting/model_best.ckpt', config["num_vertices"], config["feature_size"])

In [8]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [31]:
random_sample = random.randint(0,len(dataset))

In [32]:
random_input = dataset[random_sample]["input_sdf"]
gt_pointcloud = dataset[random_sample]["target_vertices"]
gt_mask = dataset[random_sample]["input_mask"]
gt_edges = dataset[random_sample]["target_edges"]
#faces = dataset[random_sample]["faces"]

In [33]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if gt_edges[i][j] and gt_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

In [34]:
faces = np.concatenate(faces,0)

In [35]:
visualize_pointcloud(gt_pointcloud, point_size=0.01)

Output()

In [36]:
visualize_mesh(gt_pointcloud, faces)

Output()

In [37]:
output_pointcloud, edges = inferer.infer_single(random_input, gt_mask, x_indices,y_indices,graph)

In [38]:
edges.shape

(150, 150)

In [41]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if gt_edges[i][j] and gt_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))
faces = np.concatenate(faces,0)

In [42]:
visualize_pointcloud(output_pointcloud[:int(gt_mask.sum())].squeeze(),point_size=0.01)

Output()

In [40]:
edges.sum()

149

In [41]:
gt_edges.sum()

132.0

In [43]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if edges[i][j] and edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

In [44]:
faces = np.concatenate(faces,0)

In [45]:
visualize_mesh(output_pointcloud,faces)

Output()

NameError: name 'algorithm' is not defined